In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import report

from IPython.display import display

plt.rcParams.update({'font.size': 22})

In [ ]:
df = report.pauta_com_notas()
with pd.option_context("display.max_rows", 10000):
    #display(df.drop(['chamada', 'respostas', 'gabarito'], axis=1))
    display(df[['nomecompleto', 'status', 'nota']])

Categorias:
* `noshow`: faltou a prova
* `one_attempt`: enviou exatamente uma tentativa
* `only_empty_attempts`: enviou pelo menos uma tentativa, e todas essas tentativas tinham somente questões em branco
* `no_positive_attempts`: pelo menos uma tentativa, e todas as tentativas tinham somente questões em branco ou "Nâo sei."
* `one_positive_attempt`: pelo menos uma tentativa, e exatamente uma das tentativas foi não-vazia (i.e. tinha pelo menos uma questão respondida, e a resposta não era "Não sei.")
* `lastpos_atmost1_nonpos`: pelo menos duas tentativas não-vazias, e na última das tentativas não-vazias a quantidade de questões em branco (ou "Não sei.") foi 0 ou 1.

In [ ]:
stats = df['status'].value_counts()
ax = stats.plot(kind='bar', figsize=(12, 8))
for i, (status, count) in enumerate(stats.iteritems()):
    ax.text(i, count, str(count), va='bottom', ha='center')
ax.grid()
ax.xaxis.set_tick_params(rotation=25)

# Notas

In [ ]:
delta = 10 / report.num_items
ax = df['nota'].hist(
    bins=np.linspace(-delta/2, 10 + delta/2,
                     num=report.num_items + 2),
    figsize=(12, 8))

In [ ]:
print(f"Média:  {df['nota'].mean():.3}")
print(f"Stddev: {df['nota'].std():.3}")

# Estatísticas por questão

In [ ]:
stats_respostas = []
for num_item in range(report.num_items):
    certas = 0
    erradas = 0
    naosei = 0
    submissões_válidas = 0
    for row in df.itertuples():
        if row.respostas is pd.StringDtype().na_value:
            continue
        idx = [int(x) for x in row.perm.split('-')].index(num_item)
        resposta = row.respostas.split('-')[idx]
        gabarito = row.gabarito.split('-')[idx]
        if resposta in gabarito:
            certas += 1
        if resposta != 'N' and resposta not in gabarito:
            erradas += 1
        if resposta == 'N':
            naosei += 1
        submissões_válidas += 1
    stats_respostas.append({
        'certas': certas,
        'erradas': erradas,
        'naosei': naosei,
    })
stats_respostas = pd.DataFrame(stats_respostas)
stats_respostas

In [ ]:
for num_item, row in stats_respostas.iterrows():
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax = row.plot(kind='bar', ax=ax)
    #ax.set_ylim([0, 100])
    for i, (status, count) in enumerate(row.iteritems()):
        ax.text(i, count, f"{count/submissões_válidas:.1%}",
                va='bottom', ha='center')
    ax.grid()
    ax.xaxis.set_tick_params(rotation=0)
    ax.set_title(f"Questão {num_item + 1}")

In [ ]:
row = stats_respostas.sum()
ax = row.plot(kind='bar', figsize=(12, 8))
for i, (status, count) in enumerate(row.iteritems()):
    ax.text(i, count, f"{count/submissões_válidas/report.num_items:.1%}",
            va='bottom', ha='center')
ax.grid()
ax.xaxis.set_tick_params(rotation=0)
ax.set_title(f"Total");

In [ ]:
df[['dre', 'respostas', 'gabarito', 'nota']].to_csv(
    'Notas.csv', index=False)